 ## 1. Lectura de datos

 Se comenzará con la lectura del corpus. Para lo cual se utilizarán distintas técnicas de normalización de términos, entre las que se incluye la identificación de palabras compuestas.

In [96]:
!pip install nltk
import nltk

from nltk import download

download('punkt_tab')                           # Tokenización
nltk.download('averaged_perceptron_tagger')     # POS tagging
nltk.download('averaged_perceptron_tagger_eng') # POS tagging
nltk.download('wordnet')                        # WordNet lemmatizer
nltk.download('omw-1.4')                        # WordNet multilingüe

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [97]:
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.data import path
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
import numpy as np

path.append(".")

In [98]:
!pip install contractions
import contractions

In [99]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from bs4 import MarkupResemblesLocatorWarning
import warnings

In [100]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
import spacy

In [101]:
import json
import os
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from collections import defaultdict
from tempfile import mkdtemp

!pip install whoosh
from whoosh.index import create_in
from whoosh.index import open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
from whoosh.analysis import StemmingAnalyzer
from whoosh import scoring
from whoosh.query import Term, Or

In [102]:
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [103]:
palabras_vacias_ingles = stopwords.words('english')

In [104]:
nlp = spacy.load("en_core_web_sm")

In [105]:
def elimina_html(contenido):
    return BeautifulSoup(contenido).get_text()

def elimina_no_alfanumerico(contenido):
    return [re.sub(r'[^\w]', '', palabra)
            for palabra in contenido
            if re.search(r'\w', palabra)]

def expandir_constracciones(contenido):
    return contractions.fix(contenido)

def pasar_a_minuscula(contenido):
    return contenido.lower()

def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)  # Reemplaza todo lo que no es letra o espacio con espacio
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def elimina_palabras_vacias(contenido):
    return [palabra for palabra in contenido if palabra not in palabras_vacias_ingles]

def lematizador(contenido):
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(contenido)

    resultado = []
    for palabra, tag in pos_tags:
        if tag.startswith('VB'):  # Verbos
            resultado.append(lemmatizer.lemmatize(palabra, pos='v'))  # infinitivo
        else:  # Sustantivos y el resto tal como están
            resultado.append(palabra)

    return resultado

def extraer_noun_chunks(tokens):
    resultados = []
    doc = nlp(" ".join(tokens))

    noun_chunks = [chunk.text.lower().strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
    noun_chunks_set = set(noun_chunks)

    i = 0
    while i < len(tokens):
        composed2 = " ".join(tokens[i:i+2]).lower()
 
        if composed2 in noun_chunks_set:
            i += 2 
        else:
            resultados.append(tokens[i].lower())
            i += 1

    return resultados + noun_chunks


In [106]:
def proceso_contenido(texto):
    texto = elimina_html(texto)
    texto = expandir_constracciones(texto)
    texto = pasar_a_minuscula(texto)
    texto = limpiar_texto(texto)                # Limpiar antes de tokenizar
    tokens = word_tokenize(texto)               
    tokens = elimina_no_alfanumerico(tokens)    # Limpiar tokens individuales
    tokens = elimina_palabras_vacias(tokens)
    tokens = lematizador(tokens)
    return tokens

 ## 2. Sistema de recuperación de la información que admite consultas especificadas en texto libre 

 Se procederá con el diseño y la evaluación de un sistema de recuperación de información que admite consultas en texto libre. Este se pondrá a prueba, por un lado, utilizando los documentos del corpus junto con sinónimos de las palabras contenidas en ellos, y por otro lado, sin recurrir a dichos sinónimos.

In [107]:
def procesar_csv_corpus(ruta_csv):
    """
    Carga y procesa un archivo CSV con tres columnas: autor, título y cuerpo.
    Aplica la función proceso_contenido a cada parte y devuelve una lista de listas procesadas.
    
    Parámetros:
        ruta_csv (str): Ruta al archivo CSV. Por defecto es "news_corpus.csv".
    
    Retorna:
        list: Lista donde cada elemento es una lista combinada del contenido procesado.
    """
    # Cargar CSV con codificación segura y punto y coma como separador
    df = pd.read_csv(ruta_csv, encoding="latin-1", sep=";", quotechar='"')

    # Verificar que tenga exactamente 3 columnas
    assert df.shape[1] == 3, "El CSV no tiene exactamente 3 columnas. Revisa el separador o las comillas."

    resultados = []

    for _, fila in df.iterrows():
        autor = fila.iloc[0]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]

        autor_proc = proceso_contenido(autor)
        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada = autor_proc + titulo_proc + cuerpo_proc
        resultados.append(fila_combinada)
    
    return resultados

def mostrar_documentos_procesados(resultados, n=5):
    """
    Muestra los primeros `n` documentos procesados de una lista de resultados.
    
    Parámetros:
        resultados (list): Lista de listas de palabras procesadas.
        n (int): Número de documentos a mostrar. Por defecto, 5.
    """
    for i, documento in enumerate(resultados[:n]):
        print(f"Documento {i+1}:")
        print(" - Palabras:", documento)
        print()

In [108]:
def crear_indice_whoosh_para_subcorpus(subcorpus, ruta_indice="indice"):
    """
    Crea un índice Whoosh para un subcorpus dado y lo guarda en disco.

    Parámetros:
        subcorpus (list of list of str): Lista de documentos tokenizados.
        ruta_indice (str): Carpeta donde se almacenará el índice Whoosh.
        offset (int): Valor entero que se suma al índice para generar el doc_id.
    """
    schema = Schema(doc_id=ID(stored=True),
                    content=TEXT(stored=True, analyzer=StemmingAnalyzer()))

    if not os.path.exists(ruta_indice):
        os.mkdir(ruta_indice)

    ix = create_in(ruta_indice, schema)
    writer = ix.writer()

    for i, doc in enumerate(subcorpus):
        joined_content = ' '.join(doc)
        writer.add_document(doc_id=str(i), content=joined_content)

    writer.commit()

In [109]:
def lectura_normalizada_corpus():
    df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')
    resultados = []

    for index, fila in df.iterrows():
        autor = [fila.iloc[0]]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]   

        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada =  autor + titulo_proc + cuerpo_proc

        contenido_final = extraer_noun_chunks(fila_combinada)
        resultados.append(contenido_final)
    return resultados

def expand_term(term):
    related = set()
    for syn in wn.synsets(term):
        for lemma in syn.lemmas():
            word = lemma.name().replace('_', ' ').lower()
            if word != term:
                related.add(word)
    return related

def expand_corpus_with_synonyms(documento):
    doc_counter = Counter(documento)
    expanded_doc = []
    for word, count in doc_counter.items():
        # Añadimos la palabra original tantas veces como aparece
        expanded_doc.extend([word] * count)
        # Obtenemos sinónimos y también los añadimos con la misma frecuencia
        synonyms = expand_term(word)
        for syn in synonyms:
            expanded_doc.extend([syn] * count)
    return expanded_doc

def lectura_normalizada_corpus_sinonimos():
    df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')
    resultados = []

    for index, fila in df.iterrows():
        autor = [fila.iloc[0]]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]   

        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada =  autor + titulo_proc + cuerpo_proc

        contenido_con_palabras_compuestas = extraer_noun_chunks(fila_combinada)
        contenido_final = expand_corpus_with_synonyms(contenido_con_palabras_compuestas)

        resultados.append(contenido_final)
    return resultados

def crear_indice_whoosh(corpus_normalizado):
    if os.path.exists("indice_whoosh"):
        shutil.rmtree("indice_whoosh")  # Elimina índice anterior si existe
    os.mkdir("indice_whoosh")

    schema = Schema(id=ID(stored=True, unique=True), contenido=TEXT(stored=True))
    ix = create_in("indice_whoosh", schema)
    writer = ix.writer()

    for i, doc in enumerate(corpus_normalizado):
        contenido = " ".join(doc)
        writer.add_document(id=str(i), contenido=contenido)
    
    writer.commit()


def buscar_con_whoosh(tokens_query, corpus_normalizado):
    ix = open_dir("indice_whoosh")  # Abrir índice Whoosh previamente creado

    with ix.searcher() as searcher:
        # Crear la consulta OR con todos los términos
        terms = [Term("contenido", token) for token in tokens_query]
        query = Or(terms)

        resultados = searcher.search(query, limit=None)

        # Crear diccionario: clave = doc_id, valor = documento procesado
        docs_encontrados = []
        indices_docs = []
        for hit in resultados:
            doc_id = int(hit['id'])
            docs_encontrados.append(corpus_normalizado[doc_id])
            indices_docs.append(doc_id)

        return docs_encontrados, indices_docs



In [110]:
def lectura_documento(documento):
    documento_procesado = proceso_contenido(documento) 
    contenido_final = extraer_noun_chunks(documento_procesado)
    return contenido_final

def lectura_documento_sinonimos(documento):
    documento_procesado = proceso_contenido(documento) 
    contenido_final = extraer_noun_chunks(documento_procesado)
    resultado = expand_corpus_with_synonyms(contenido_final)
    return resultado

In [111]:
# Mostrar los primeros 3 documentos procesados
def prueba_primeros_3_documentos_procesados(corpus):
    for i, documento in enumerate(corpus[:3]):
        print(f"Documento {i+1}:")
        print(" - Palabras:", documento)
        print()

In [112]:
def tfidf_del_documento(documento_normalizado, vectorizer):
    # Convertir el documento (lista de tokens) a string
    texto = " ".join(documento_normalizado)
    # Transformar usando el vectorizador ya entrenado
    X_doc = vectorizer.transform([texto])  # devuelve matriz sparse 1xN
    return X_doc

In [113]:
def prueba_tfdifs_primeros_3_documentos(lista_diccionarios_tfidfs):
    for idx, d in enumerate(lista_diccionarios_tfidfs[:3]):
        top_terms = sorted(d.items(), key=lambda x: x[1], reverse=True)
        print(f"\n Documento {idx+1}:")
        print("      Palabras añadidas por TF-IDF:")
        for term, score in top_terms:
            print(f"      - {term}: {score:.4f}")

In [114]:
def tfidf_por_corpus(corpus_normalizado):
    # Convertimos el corpus a lista de strings
    texts = [" ".join(doc) for doc in corpus_normalizado]

    # Vectorizador TF-IDF (1 y 2-gramas)
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X = vectorizer.fit_transform(texts)  # Matriz TF-IDF sparse
    terms = vectorizer.get_feature_names_out()

    # Retornamos la matriz y vocabulario (términos)
    return X, terms, vectorizer

In [115]:
def similitud_coseno(tfidf_corpus, tfidf_doc, indices, umbral=0.0):
    # Calcular similitud coseno entre documento y corpus
    similitudes = cosine_similarity(tfidf_doc, tfidf_corpus).flatten()

    # Filtrar documentos que superan el umbral
    indices_filtrados = [i for i, sim in enumerate(similitudes) if sim > umbral]

    # Ordenar índices por similitud descendente
    indices_ordenados = sorted(indices_filtrados, key=lambda i: similitudes[i], reverse=True)

    # Devolver lista de (indice, similitud)
    return [(indices[i], similitudes[i]) for i in indices_ordenados]

def documentos_similares(lista_similitudes):
    lista_similitudes[0]
    for idx, score in lista_similitudes:
        print(f"Documento {idx} tiene similitud: {score:.4f}")
    

In [116]:
# Cargar CSV con codificación segura y punto y coma como separador
ruta_csv="news_corpus.csv"

resultados = procesar_csv_corpus(ruta_csv)
mostrar_documentos_procesados(resultados)

Documento 1:
 - Palabras: ['chhavi', 'tyagi', 'daman', 'diu', 'revoke', 'mandatory', 'rakshabandhan', 'offices', 'order', 'daman', 'diu', 'administration', 'wednesday', 'withdraw', 'circular', 'ask', 'women', 'staff', 'tie', 'rakhis', 'male', 'colleagues', 'order', 'trigger', 'backlash', 'employees', 'rip', 'apart', 'social', 'media', 'union', 'territory', 'administration', 'force', 'retreat', 'within', 'hours', 'issue', 'circular', 'make', 'compulsory', 'staff', 'celebrate', 'rakshabandhan', 'workplace', 'decide', 'celebrate', 'festival', 'rakshabandhan', 'august', 'connection', 'offices', 'departments', 'shall', 'remain', 'open', 'celebrate', 'festival', 'collectively', 'suitable', 'time', 'wherein', 'lady', 'staff', 'shall', 'tie', 'rakhis', 'colleagues', 'order', 'issue', 'august', 'gurpreet', 'singh', 'deputy', 'secretary', 'personnel', 'say', 'ensure', 'one', 'skipped', 'office', 'attendance', 'report', 'send', 'government', 'next', 'even', 'two', 'notifications', 'one', 'mandate

In [117]:
# Prueba de lectura 
resultados_lectura = lectura_normalizada_corpus()
prueba_primeros_3_documentos_procesados(resultados_lectura)

Documento 1:
 - Palabras: ['diu', 'revoke', 'mandatory', 'rakshabandhan', 'offices', 'order', 'daman', 'wednesday', 'withdraw', 'circular', 'ask', 'tie', 'rakhis', 'male', 'colleagues', 'order', 'trigger', 'backlash', 'employees', 'rip', 'apart', 'social', 'media', 'union', 'territory', 'administration', 'force', 'retreat', 'within', 'circular', 'make', 'celebrate', 'decide', 'celebrate', 'festival', 'rakshabandhan', 'august', 'connection', 'offices', 'departments', 'shall', 'remain', 'open', 'celebrate', 'festival', 'collectively', 'suitable', 'time', 'wherein', 'shall', 'tie', 'rakhis', 'colleagues', 'order', 'issue', 'august', 'gurpreet', 'singh', 'deputy', 'secretary', 'personnel', 'say', 'ensure', 'one', 'skipped', 'office', 'attendance', 'report', 'send', 'government', 'next', 'even', 'two', 'notifications', 'one', 'mandate', 'celebration', 'rakshabandhan', 'leave', 'withdraw', 'mandate', 'daman', 'day', 'apart', 'circular', 'withdrawn', 'one', 'line', 'order', 'issue', 'late', '

In [118]:
def cargar_consultas_booleanas_especificas(archivo):
    """
    Carga un archivo JSON con consultas booleanas específicas.

    Parámetros:
        archivo (str): Ruta al archivo JSON.

    Retorna:
        list: Lista de consultas cargadas desde el archivo.
    """
    with open(archivo, "r", encoding="utf-8") as f:
        consultas = json.load(f)
    return consultas

def verificar_consultas(consultas):
    """
    Imprime el número total de consultas cargadas y muestra las tres primeras.

    Parámetros:
        consultas (list): Lista de consultas, cada una con los campos 'tema', 'query' y 'documentos'.
    """
    print(f"\nNúmero total de consultas cargadas: {len(consultas)}\n")

    for i, consulta in enumerate(consultas[:3]):
        print(f"Consulta {i+1}:")
        print(f"  Tema: {consulta.get('tema', 'N/A')}")
        print(f"  Query: {consulta.get('query', 'N/A')}")
        print(f"  Documentos relevantes: {consulta.get('documentos', [])}")
        print("-" * 60)

In [119]:
import json

def cargar_consultas(nombre_archivo):
    """Carga las consultas desde un archivo JSON y las devuelve."""
    with open(nombre_archivo, "r", encoding="utf-8") as f:
        consultas = json.load(f)
    return consultas

def mostrar_consultas_ejemplo(consultas, n=2):
    """Muestra las primeras `n` consultas para verificación."""
    print(f"Se han cargado {len(consultas)} consultas.")
    for i in range(min(n, len(consultas))):
        print(f"Consulta {i+1}:", consultas[i])


In [120]:
# 2.3. Cargar las consultas booleanas

archivo="consultas_booleanas_especificas.txt"

consultas_booleanas = cargar_consultas_booleanas_especificas(archivo)
verificar_consultas(consultas_booleanas)


Número total de consultas cargadas: 59

Consulta 1:
  Tema: Imposición de celebraciones religiosas en entornos laborales
  Query: rakshabandhan AND (office OR offices) AND circular
  Documentos relevantes: [0]
------------------------------------------------------------
Consulta 2:
  Tema: Controversias sobre normas laborales y religión
  Query: government AND festival AND mandatory AND workplace
  Documentos relevantes: [0]
------------------------------------------------------------
Consulta 3:
  Tema: Críticas a mujeres famosas tras el divorcio
  Query: malaika AND divorce AND troll
  Documentos relevantes: [1]
------------------------------------------------------------


In [121]:
# 2.4. Cargar las consultas

nombre_archivo="consultas_texto_libre.json"

consultas = cargar_consultas(nombre_archivo)
mostrar_consultas_ejemplo(consultas)

Se han cargado 66 consultas.
Consulta 1: {'query': 'News about Donald Trump', 'documentos': [32, 33, 35, 37, 40, 43]}
Consulta 2: {'query': 'Information about films, series and shows', 'documentos': [41, 46, 47, 48]}


In [122]:
from sklearn.metrics import average_precision_score

def calcular_metricas(resultados_similitud, documentos_relevantes, indices_posibles):
    y_true_positive = []
    y_false_positive = []
    y_scores = []
    idxs = []

    for idx, score in resultados_similitud:
        y_true_positive.append(1 if idx in documentos_relevantes else 0)
        y_false_positive.append(1 if idx not in documentos_relevantes else 0)
        y_scores.append(score)
        idxs.append(idx)

    no_relevantes = [i for i in indices_posibles if i not in idxs]
    y_true_negative = [1 if i not in documentos_relevantes else 0 for i in no_relevantes]
    y_false_negative = [1 if i in documentos_relevantes else 0 for i in no_relevantes]

    if any(y_true_positive):
        ap = average_precision_score(y_true_positive, y_scores)
        precision = sum(y_true_positive) / (sum(y_true_positive) + sum(y_false_positive))
        recall = sum(y_true_positive) / (sum(y_true_positive) + sum(y_false_negative))
        return ap, precision, recall, idxs, y_scores
    else:
        return None, None, None, idxs, y_scores


def mostrar_resultados(texto_query, documentos_relevantes, resultados_similitud, ap, precision, recall):
    print(f"Consulta: {texto_query}")
    print(f"Documentos relevantes esperados: {sorted(documentos_relevantes)}")
    print(f"Documentos recuperados ordenados: {[i for i, _ in resultados_similitud]}")
    print(f"Average Precision: {ap:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("-" * 60)


def evaluar_consultas_por_similitud(consultas, corpus_normalizado):
    crear_indice_whoosh(corpus_normalizado)
    map_scores = []
    map_precision = []
    map_recall = []
    #indices_posibles = list(range(len(corpus_normalizado)))
    indices_posibles = list(range(51))

    for consulta in consultas:
        texto_query = consulta["query"]
        documentos_relevantes = set(consulta["documentos"])

        lectura = lectura_documento(texto_query)
        docs_relevantes, indices_docs = buscar_con_whoosh(lectura, corpus_normalizado)
        tfidf_corpus, terms, vectorizer = tfidf_por_corpus(docs_relevantes)
        tfidf_documento = tfidf_del_documento(lectura, vectorizer)
        resultados_similitud = similitud_coseno(tfidf_corpus, tfidf_documento, indices_docs)

        # Lógica de métricas
        ap, precision, recall, idxs, y_scores = calcular_metricas(resultados_similitud, documentos_relevantes, indices_posibles)

        if ap is not None:
            map_scores.append(ap)
            map_precision.append(precision)
            map_recall.append(recall)

            mostrar_resultados(texto_query, documentos_relevantes, resultados_similitud, ap, precision, recall)
        else:
            print(f"Consulta: {texto_query}")
            print("No hay documentos relevantes etiquetados para esta consulta. Se omite en MAP.")
            print("-" * 60)

    return map_scores, map_precision, map_recall

In [123]:
# 2.1.Preparar los datos del corpus

subcorpus = resultados[0:30]

In [124]:
# 2.2. Definir el esquema de Whoosh e indexar los documentos

crear_indice_whoosh_para_subcorpus(subcorpus)

In [125]:
def calcular_metricas_booleanos(doc_encontrados, doc_relevantes):
    """
    Calcula precisión y recall para una consulta.

    Retorna:
        (precision, recall)
    """
    verdaderos_positivos = doc_encontrados & doc_relevantes
    precision = len(verdaderos_positivos) / len(doc_encontrados) if doc_encontrados else 0
    recall = len(verdaderos_positivos) / len(doc_relevantes) if doc_relevantes else 0
    return precision, recall

def imprimir_metricas_individuales(tema, consulta, encontrados, relevantes, precision, recall):
    """
    Imprime las métricas de una consulta específica.
    """
    print(f"Tema: {tema}")
    print(f"Consulta: {consulta}")
    print(f"Documentos encontrados: {sorted(encontrados)}")
    print(f"Documentos relevantes: {sorted(relevantes)}")
    print(f"Precisión: {precision:.2f}")
    print(f"Recall (Sensibilidad): {recall:.2f}")
    print("-" * 60)

from whoosh.qparser import QueryParser
from whoosh.index import open_dir
from whoosh import scoring

def ejecutar_y_evaluar_consultas(consultas, indice_path):
    """
    Ejecuta todas las consultas usando el índice Whoosh y evalúa precisión y recall.

    Parámetros:
        consultas (list): Lista de consultas con 'query', 'documentos' y 'tema'.
        indice_path (str): Ruta al directorio del índice Whoosh.
    """
    ix = open_dir(indice_path)
    parser = QueryParser("content", ix.schema)

    total_precision = 0
    total_recall = 0
    num_consultas = len(consultas)

    with ix.searcher(weighting=scoring.BM25F()) as searcher:
        for consulta in consultas:
            query_text = consulta["query"]
            documentos_relevantes = set(map(str, consulta["documentos"]))
            tema = consulta["tema"]

            try:
                query = parser.parse(query_text)
                resultados = searcher.search(query, limit=None)
                documentos_encontrados = set([r['doc_id'] for r in resultados])
            except Exception as e:
                print(f"Error al procesar la consulta '{query_text}': {e}")
                continue

            precision, recall = calcular_metricas_booleanos(documentos_encontrados, documentos_relevantes)
            total_precision += precision
            total_recall += recall

            imprimir_metricas_individuales(tema, query_text, documentos_encontrados, documentos_relevantes, precision, recall)

    return total_precision, total_recall, num_consultas

In [126]:
# 2.4. Ejecutar las consultas y evaluar

indice_path="indice"
total_precision, total_recall, num_consultas = ejecutar_y_evaluar_consultas(consultas_booleanas, indice_path)

Tema: Imposición de celebraciones religiosas en entornos laborales
Consulta: rakshabandhan AND (office OR offices) AND circular
Documentos encontrados: ['0']
Documentos relevantes: ['0']
Precisión: 1.00
Recall (Sensibilidad): 1.00
------------------------------------------------------------
Tema: Controversias sobre normas laborales y religión
Consulta: government AND festival AND mandatory AND workplace
Documentos encontrados: ['0']
Documentos relevantes: ['0']
Precisión: 1.00
Recall (Sensibilidad): 1.00
------------------------------------------------------------
Tema: Críticas a mujeres famosas tras el divorcio
Consulta: malaika AND divorce AND troll
Documentos encontrados: ['1']
Documentos relevantes: ['1']
Precisión: 1.00
Recall (Sensibilidad): 1.00
------------------------------------------------------------
Tema: Misoginia en redes sociales hacia mujeres independientes
Consulta: woman AND successful AND troll AND gym
Documentos encontrados: ['1']
Documentos relevantes: ['1']
Pre

In [127]:
#Calculo de similitudes para cada consulta sin hacer uso de sinónimos

corpus_normalizado = lectura_normalizada_corpus()
corpus_normalizado_reducido = corpus_normalizado[0:51]
map_scores, map_precision, map_recall = evaluar_consultas_por_similitud(consultas, corpus_normalizado_reducido)

Consulta: News about Donald Trump
Documentos relevantes esperados: [32, 33, 35, 37, 40, 43]
Documentos recuperados ordenados: [32, 35, 37, 40, 33, 43, 9, 16, 19, 20, 11, 2, 36]
Average Precision: 1.0000
Precision: 0.4615
Recall: 1.0000
------------------------------------------------------------
Consulta: Information about films, series and shows
Documentos relevantes esperados: [41, 46, 47, 48]
Documentos recuperados ordenados: [47, 34, 46, 37, 41, 28, 11, 23, 33, 29]
Average Precision: 0.7556
Precision: 0.3000
Recall: 0.7500
------------------------------------------------------------
Consulta: Cases related to racism and hindu culture in offices
Documentos relevantes esperados: [42]
Documentos recuperados ordenados: [0, 42, 50, 19, 20, 7, 45, 25, 36, 4]
Average Precision: 0.5000
Precision: 0.1000
Recall: 1.0000
------------------------------------------------------------
Consulta: Data on economy and finance
Documentos relevantes esperados: [44, 45]
Documentos recuperados ordenados: